## import necessary libraries for pruning

In [1]:

import torch
import torch.nn as nn
import torch_pruning as tp

from torch import optim
from torch_pruning.pruner.algorithms.scheduler import linear_scheduler
from torchsummary import summary

from cnn.resNet.resnet_example import get_data_loaders
import torch
from torch import nn

### Seed Network

In [2]:
class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expansion=6):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        self.use_res_connect = self.stride == 1 and in_channels == out_channels

        # Standard PyTorch layers (NO torch_pruning wrappers needed)
        self.conv1 = nn.Conv2d(in_channels, in_channels * expansion, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels * expansion)
        self.relu = nn.ReLU6(inplace=True)

        self.conv2 = nn.Conv2d(
            in_channels * expansion, in_channels * expansion, kernel_size=3,
            stride=stride, padding=1, groups=in_channels * expansion, bias=False
        )
        self.bn2 = nn.BatchNorm2d(in_channels * expansion)

        self.conv3 = nn.Conv2d(in_channels * expansion, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        identity = x if self.use_res_connect else None

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.use_res_connect:
            return identity + out
        else:
            return out

### Mask Network

In [3]:
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=10):
        super(MobileNetV2, self).__init__()
        # --- Remove mask-related parameters ---
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU6(inplace=True)

        # Define blocks (no mask_index or mask)
        self.block1 = InvertedResidual(32, 16, stride=1)
        self.block2 = InvertedResidual(16, 24, stride=2)
        self.block3 = InvertedResidual(24, 32, stride=2)
        self.block4 = InvertedResidual(32, 64, stride=2)
        self.block5 = InvertedResidual(64, 96, stride=1)
        self.block6 = InvertedResidual(96, 160, stride=2)
        self.block7 = InvertedResidual(160, 320, stride=1)

        self.conv2 = nn.Conv2d(320, 1280, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(1280)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        # --- Remove mask-based block skipping ---
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def save_model_as_onnx(model, example_input, output_path):
    # Set model to evaluation mode
    model.eval()
    # Export to ONNX
    torch.onnx.export(
        model,
        example_input,
        output_path,
        export_params=True,
        opset_version=13,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    print(f"✅ Model saved as ONNX to {output_path}")

In [6]:
def calculate_macs(model, example_input):
    macs, params = tp.utils.count_ops_and_params(model, example_input)
    return macs, params

### compare results of different pruning strategies

In [7]:
def compare_results(results):
    print("\n=== Pruning Strategy Comparison ===")
    print(f"{'Strategy':<12} | {'MACs':<12} | {'Size (MB)':<10} | {'Accuracy (%)':<12}")
    print("-" * 55)
    for strategy, metrics in results.items():
        print(f"{strategy:<12} | {metrics['macs']:.2e} | {metrics['size_mb']:>9.2f} | {metrics['accuracy']:>12.2f}")

### Load Model

In [8]:
def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model

### Utility function to save the model

In [9]:
import os

def save_model(model, path, example_input=None):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(model.state_dict(), path)
    if example_input is not None:
        onnx_path = path.replace('.pth', '.onnx')
        save_model_as_onnx(model, example_input, onnx_path)

### Evaluate the model

In [24]:
def evaluate_model(model, test_loader, example_input, device):
    model.eval()
    # Calculate metrics
    macs, _ = calculate_macs(model, example_input)
    params = sum(p.numel() for p in model.parameters())
    size_mb = params * 4 / 1e6

    # Calculate accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = [d.to(device) for d in data]
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return {
        'macs': macs,
        'size_mb': size_mb,
        'accuracy': 100 * correct / total
    }

### Prune the model

In [19]:
def prune_model(model, example_input, target_macs, strategy, iterative_steps=5, iterative_pruning_ratio_scheduler=linear_scheduler()):
    if isinstance(strategy['importance'], tp.importance.TaylorImportance):
        pruning_ratio = 0.1
    else:
        pruning_ratio = 0.5

    pruner = strategy['pruner'](
        model,
        example_input,
        importance=strategy['importance'],
        iterative_steps=iterative_steps,
        ch_sparsity=pruning_ratio,  # Initial sparsity
        iterative_pruning_ratio_scheduler=iterative_pruning_ratio_scheduler,
        root_module_types=[nn.Conv2d],
        ignored_layers=[model.fc],
    )

    current_macs, base_nparams = calculate_macs(model, example_input)
    # while current_macs > target_macs:
    #     pruner.step()
    #     current_macs = calculate_macs(model, example_input)


    for i in range(iterative_steps):
            if isinstance(strategy['importance'], tp.importance.TaylorImportance):
                loss = model(example_input).sum() # a dummy loss for TaylorImportance
                loss.backward()
            for g in pruner.step(interactive=True):
                g.prune()
            macs, nparams = tp.utils.count_ops_and_params(model, example_input)
            print(model(example_input).shape)
            print(
                "  Iter %d/%d, Params: %.2f M => %.2f M"
                % (i + 1, iterative_steps, base_nparams / 1e6, nparams / 1e6)
            )
            print(
                "  Iter %d/%d, MACs: %.2f G => %.2f G"
                % (i + 1, iterative_steps, current_macs / 1e9, macs / 1e9)
            )

    return model

### Train the model

In [4]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            inputs, labels = [d.to(device) for d in data]
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}: Loss={epoch_loss:.4f}, Accuracy={epoch_acc:.2f}%")
    return model

### Main workflow

In [22]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Configuration
    config = {
        'strategies': {
            'magnitude': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.MagnitudeImportance(p=2),
            },
            'bn_scale': {
                'pruner': tp.pruner.BNScalePruner,
                'importance': tp.importance.BNScaleImportance(),
            },
            # todo: check the examples for the following strategies, why it is giving error
            'group_norm': {
                'pruner': tp.pruner.GroupNormPruner,
                'importance': tp.importance.GroupMagnitudeImportance(p=1),
            },
            'random': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.RandomImportance(),
            },
            'Taylor': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.TaylorImportance()
            },
            'Hessian': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.GroupHessianImportance()
            },
            'lamp': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.LAMPImportance(p=2)
            },
            #todo: implement growing reg pruning
        },
        #todo: different types of schedulers can be added
        'iterative_pruning_ratio_scheduler': linear_scheduler(),
        'target_macs_sparsity': 0.5,
        'train_epochs': 10,
        'fine_tune_epochs': 20,
        'data_dir': './data',
        'output_dir': './output/strategies',
        'iterative_steps': 5,
    }

    # Initialize model and data
    model = MobileNetV2(num_classes=10).to(device)
    train_loader, test_loader = get_data_loaders(config['data_dir'])
    example_input = torch.randn(1, 3, 32, 32).to(device)

    # Workflow execution
    initial_model_path = os.path.join(config['output_dir'], "mobilenetv2_initial.pth")

    if not os.path.exists(initial_model_path):
        # 1. Initial training
        model = train_model(
            model=model,
            train_loader=train_loader,
            criterion=nn.CrossEntropyLoss().to(device),
            optimizer=optim.Adam(model.parameters(), lr=0.001),
            device=device,
            num_epochs=config['train_epochs']
        )
        save_model(model, initial_model_path, example_input)

    # 2. Pruning and evaluation workflow
    results = {}
    initial_macs, initial_params = calculate_macs(model, example_input)
    target_macs = initial_macs * config['target_macs_sparsity']

    for strategy_name in config['strategies']:
        # 2a. Prepare fresh model for each strategy
        model_copy = load_model(MobileNetV2(num_classes=10).to(device), initial_model_path)

        # 2b. Perform pruning
        pruned_model = prune_model(
            model=model_copy,
            example_input=example_input,
            target_macs=target_macs,
            strategy=config['strategies'][strategy_name],
            iterative_steps=config['iterative_steps'],
            'iterative_pruning_ratio_scheduler'=config['iterative_pruning_ratio_scheduler'],
        )

        # 2c. Save pruned model
        pruned_path = os.path.join(config['output_dir'], f"mobilenetv2_{strategy_name}_pruned.pth")
        save_model(pruned_model, pruned_path, example_input)

        # 2d. Fine-tune
        # todo: lr, more epochs, early stopping
        fine_tuned_model = train_model(
            model=pruned_model,
            train_loader=train_loader,
            criterion=nn.CrossEntropyLoss().to(device),
            optimizer=optim.Adam(pruned_model.parameters(), lr=0.001),
            device=device,
            num_epochs=config['fine_tune_epochs']
        )

        # 2e. Evaluate
        results[strategy_name] = evaluate_model(
            model=fine_tuned_model,
            test_loader=test_loader,
            example_input=example_input,
            device=device
        )

        # 2f. Save final model
        final_path = os.path.join(config['output_dir'], f"mobilenetv2_{strategy_name}_final.pth")
        save_model(fine_tuned_model, final_path, example_input)

    # 3. Compare results
    compare_results(results)
    print("Workflow completed successfully!")

In [25]:
if __name__ == "__main__":
    main()

Using dataset directory: /home/muis/thesis/github-repo/master-thesis/cnn/mobile_net_v2/data
Epoch 1/10: Loss=1.6531, Accuracy=39.32%
Epoch 2/10: Loss=1.2808, Accuracy=54.42%
Epoch 3/10: Loss=1.0993, Accuracy=61.08%
Epoch 4/10: Loss=0.9806, Accuracy=65.62%
Epoch 5/10: Loss=0.8908, Accuracy=69.04%
Epoch 6/10: Loss=0.8198, Accuracy=71.39%
Epoch 7/10: Loss=0.7561, Accuracy=73.34%
Epoch 8/10: Loss=0.7010, Accuracy=75.39%
Epoch 9/10: Loss=0.6570, Accuracy=77.17%
Epoch 10/10: Loss=0.6133, Accuracy=78.39%
✅ Model saved as ONNX to ./output/strategies/mobilenetv2_initial.onnx
torch.Size([1, 10])
  Iter 1/5, Params: 1.17 M => 0.95 M
  Iter 1/5, MACs: 0.01 G => 0.00 G
torch.Size([1, 10])
  Iter 2/5, Params: 1.17 M => 0.76 M
  Iter 2/5, MACs: 0.01 G => 0.00 G
torch.Size([1, 10])
  Iter 3/5, Params: 1.17 M => 0.58 M
  Iter 3/5, MACs: 0.01 G => 0.00 G
torch.Size([1, 10])
  Iter 4/5, Params: 1.17 M => 0.43 M
  Iter 4/5, MACs: 0.01 G => 0.00 G
torch.Size([1, 10])
  Iter 5/5, Params: 1.17 M => 0.31 M
  

### Load the saved Onnx model and convert to Pytorch model

In [12]:
import torch
import onnx
import onnxruntime
from onnx2torch import convert

# Step 1: Load the ONNX model
onnx_model_path = './output/strategies/mobilenetv2_bn_scale_final.onnx'
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)  # Verify the ONNX model
print("✅ ONNX model loaded and verified.")

# Step 2: Convert ONNX to PyTorch
torch_model = convert(onnx_model)
print("✅ ONNX model converted to PyTorch.")
tp.utils.print_tool.before_pruning(torch_model)


✅ ONNX model loaded and verified.
✅ ONNX model converted to PyTorch.


In [13]:
from torchinfo import summary
print(summary(torch_model))

Layer (type:depth-idx)                                  Param #
GraphModule                                             --
├─Conv2d: 1-1                                           448
├─OnnxConstant: 1-2                                     --
├─OnnxConstant: 1-3                                     --
├─ReLU6: 1-4                                            --
├─Conv2d: 1-5                                           1,632
├─OnnxConstant: 1-6                                     --
├─OnnxConstant: 1-7                                     --
├─ReLU6: 1-8                                            --
├─Conv2d: 1-9                                           960
├─OnnxConstant: 1-10                                    --
├─OnnxConstant: 1-11                                    --
├─ReLU6: 1-12                                           --
├─Conv2d: 1-13                                          776
├─Conv2d: 1-14                                          432
├─OnnxConstant: 1-15                        

In [11]:

from torchinfo import summary
print(summary(torch_model))
#tp.utils.print_tool.after_pruning(torch_model)


Layer (type:depth-idx)                                  Param #
GraphModule                                             --
├─Conv2d: 1-1                                           896
├─OnnxConstant: 1-2                                     --
├─OnnxConstant: 1-3                                     --
├─ReLU6: 1-4                                            --
├─Conv2d: 1-5                                           6,336
├─OnnxConstant: 1-6                                     --
├─OnnxConstant: 1-7                                     --
├─ReLU6: 1-8                                            --
├─Conv2d: 1-9                                           1,920
├─OnnxConstant: 1-10                                    --
├─OnnxConstant: 1-11                                    --
├─ReLU6: 1-12                                           --
├─Conv2d: 1-13                                          3,088
├─Conv2d: 1-14                                          1,632
├─OnnxConstant: 1-15                  

### Load the converted Pytorch model and fine-tune

In [6]:
train_loader, test_loader = get_data_loaders('./data')
# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_model = torch_model.to(device)

tp.utils.print_tool.before_pruning(torch_model)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(torch_model.parameters(), lr=0.001)
num_epochs = 50

fine_tuned_model = train_model(torch_model, train_loader, criterion, optimizer, device, num_epochs)

Using dataset directory: /home/muis/thesis/github-repo/master-thesis/cnn/mobile_net_v2/data
Epoch 1/50: Loss=0.6039, Accuracy=78.76%
Epoch 2/50: Loss=0.6009, Accuracy=78.70%
Epoch 3/50: Loss=0.5960, Accuracy=78.91%
Epoch 4/50: Loss=0.5975, Accuracy=78.81%
Epoch 5/50: Loss=0.5957, Accuracy=78.92%
Epoch 6/50: Loss=0.5969, Accuracy=78.82%
Epoch 7/50: Loss=0.6109, Accuracy=78.42%
Epoch 8/50: Loss=0.6080, Accuracy=78.65%
Epoch 9/50: Loss=0.6081, Accuracy=78.52%
Epoch 10/50: Loss=0.6090, Accuracy=78.53%
Epoch 11/50: Loss=0.6294, Accuracy=77.73%
Epoch 12/50: Loss=0.6354, Accuracy=77.75%
Epoch 13/50: Loss=0.6369, Accuracy=77.76%
Epoch 14/50: Loss=0.6398, Accuracy=77.80%
Epoch 15/50: Loss=0.6449, Accuracy=77.49%
Epoch 16/50: Loss=0.6451, Accuracy=77.21%
Epoch 17/50: Loss=0.6569, Accuracy=76.93%
Epoch 18/50: Loss=0.6594, Accuracy=76.96%
Epoch 19/50: Loss=0.6644, Accuracy=76.68%
Epoch 20/50: Loss=0.6890, Accuracy=76.04%
Epoch 21/50: Loss=0.7159, Accuracy=75.11%
Epoch 22/50: Loss=0.7258, Accuracy=